1. pair representations $h_v^{(0)}$  赋值子图中每个节点和源节点信息
2. Message $M_v^{(1)}$: {$h_v^{(0)}$, 所有指向v的节点x的$h_x$和(x, v)总结后的信息}
3. 总结所有Message

In [1]:
import pandas as pd

In [2]:
sample = pd.read_csv('sample.csv')
cite = pd.read_csv('sample_cite.csv')

In [3]:
# 第一步
src = sample[sample.pid == 10169]
tar = sample[sample.pid == 63486]
v_df = sample[sample.pid != src.pid.values[0]].reset_index(drop=True)

h_v = pd.DataFrame({
    'src_pid': [src.pid.values[0]] * len(v_df),
    'src_title': [src.title.values[0]] * len(v_df),
    'src_abs': [src['abs'].values[0]] * len(v_df),
    'src_label': [src.label.values[0]] * len(v_df),
    'v_pid': v_df.pid,
    'v_title': v_df.title,
    'v_abs': v_df['abs'],
    'v_label': v_df.label
})
h_v

,src_pid,src_title,src_abs,src_label,v_pid,v_title,v_abs,v_label
0,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,63486,Title: Evaluation and Selection of Biases in M...,"Abstract: In this introduction, we define the ...",Theory
1,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,2440,"Title: Quinlan, 1990 J.R. Quinlan. Learning lo...",Abstract: We describe a ranked-model semantics...,Rule_Learning
2,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,3231,Title: Irrelevant Features and the Subset Sele...,Abstract: We address the problem of finding a ...,Theory
3,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,954315,Title: Inductive Database Design,Abstract: When designing a (deductive) databas...,Rule_Learning
4,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,144330,Title: Utilizing Prior Concepts for Learning,Abstract: The inductive learning problem consi...,Theory
5,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,56115,Title: Improving Tactical Plans with Genetic A...,Abstract:,Genetic_Algorithms
6,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,72908,Title: Applications of a logical discovery engine,Abstract: The clausal discovery engine claudie...,Rule_Learning
7,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,83461,Title: Dlab: A Declarative Language Bias Forma...,Abstract: We describe the principles and funct...,Rule_Learning
8,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,17811,Title: #1 Robust Feature Selection Algorithms,Abstract: Selecting a set of features which is...,Genetic_Algorithms
9,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,37483,Title: Learning Approximate Control Rules Of H...,Abstract: One of the difficult problems in the...,Case_Based


In [4]:
src.pid, tar['pid'].values[0]

(3    10169
 Name: pid, dtype: int64,
 63486)

In [5]:
# Indicator

hv_list_orig = []
for i in range(len(h_v)):

    src_title, src_abs, src_label = h_v.loc[i, 'src_title'], h_v.loc[i, 'src_abs'], h_v.loc[i, 'src_label']
    tar_pid, tar_title, tar_abs, tar_label = h_v.loc[i, 'v_pid'], h_v.loc[i, 'v_title'], h_v.loc[i, 'v_abs'], h_v.loc[i, 'v_label']

    # if tar['pid'].values[0] == tar_pid:
    #     continue
    
    # print(tar_pid)

    Indicator_prompt = f"""
Given information of a source paper and a target paper shown as follow:
source paper:
title:{src_title}
abstract:{src_abs}
label:{src_label}
target paper:
title:{tar_title}
abstract:{tar_abs}
label:{tar_label}
please summarize information of these two papers in no more than three sentences, and then generate some keywords of this information.
When outputting your answer, when referring to a paper, you should state the title of the paper and indicate whether it is a source paper or a target paper.
"""

    hv_list_orig.append(Indicator_prompt)

In [6]:
for hv in hv_list_orig:
    print(hv)
    print('======================================================================================')


Given information of a source paper and a target paper shown as follow:
source paper:
title:Title: Learning Boolean Concepts in the Presence of Many Irrelevant Features
abstract:Abstract: In this paper, we address the problem of case-based learning in the presence of irrelevant features. We review previous work on attribute selection and present a new algorithm, Oblivion, that carries out greedy pruning of oblivious decision trees, which effectively store a set of abstract cases in memory. We hypothesize that this approach will efficiently identify relevant features even when they interact, as in parity concepts. We report experimental results on artificial domains that support this hypothesis, and experiments with natural domains that show improvement in some cases but not others. In closing, we discuss the implications of our experiments, consider additional work on irrelevant features, and outline some directions for future research.
label:Theory
target paper:
title:Title: Evaluati

In [7]:
# w(x, r, v)
# cite, src, tar
w_list_orig = []
for i in range(len(cite)):
    src_id, tar_id = cite.loc[i, 'src_id'], cite.loc[i, 'tar_id']

    if src.pid.values[0] == src_id and tar.pid.values[0] == tar_id:
        # print(src_id, tar_id)
        continue
    
    src_title, src_abs, src_label = sample[sample.pid == src_id].title.values[0], sample[sample.pid == src_id]['abs'].values[0], sample[sample.pid == src_id].label.values[0]
    tar_title, tar_abs, tar_label = sample[sample.pid == tar_id].title.values[0], sample[sample.pid == tar_id]['abs'].values[0], sample[sample.pid == tar_id].label.values[0]
    

    w_prompt = f"""
{src_id} {tar_id}
paper1:
title:{src_title}
label:{src_label}
cites
paper2:
title:{tar_title}
label:{tar_label}
    
"""

    w_list_orig.append(w_prompt)

for w in w_list_orig:
    print(w)
    print('=========================================================================')


2440 72908
paper1:
title:Title: Quinlan, 1990 J.R. Quinlan. Learning logical definitions from relations. Machine Learning, First-order theory revision. In
label:Rule_Learning
cites
paper2:
title:Title: Applications of a logical discovery engine
label:Rule_Learning
    


3231 63486
paper1:
title:Title: Irrelevant Features and the Subset Selection Problem
label:Theory
cites
paper2:
title:Title: Evaluation and Selection of Biases in Machine Learning
label:Theory
    


10169 3231
paper1:
title:Title: Learning Boolean Concepts in the Presence of Many Irrelevant Features
label:Theory
cites
paper2:
title:Title: Irrelevant Features and the Subset Selection Problem
label:Theory
    


17811 63486
paper1:
title:Title: #1 Robust Feature Selection Algorithms
label:Genetic_Algorithms
cites
paper2:
title:Title: Evaluation and Selection of Biases in Machine Learning
label:Theory
    


37483 2440
paper1:
title:Title: Learning Approximate Control Rules Of High Utility
label:Case_Based
cites
paper2:

In [8]:
# message

src_id = src.pid.values[0]
tar_id = tar.pid.values[0]

message = {}
for i in range(len(sample)):

    node_id = sample.loc[i, 'pid']

    if node_id == src_id:
        continue


    h0 = sample.loc[i, 'h0']
    message[node_id] = {'h': [h0.replace('\n', '')]}

message

w
for key in message.keys():
    x_df = cite[cite.tar_id == key].reset_index(drop = True)
    message[key]['w'] = []
    for i in range(len(x_df)):
        if x_df.loc[i, 'src_id'] == src_id and x_df.loc[i, 'tar_id'] == tar_id:
            continue

        message[key]['w'].append(x_df.loc[i, 'w'].replace('\r', '').replace('\n', ''))


message
    

{63486: {'h': ['The source paper, "Learning Boolean Concepts in the Presence of Many Irrelevant Features," explores addressing case-based learning amidst irrelevant features, proposing the Oblivion algorithm for efficient feature identification, especially with interacting features like parity concepts. It discusses experimental results in artificial domains supporting its hypothesis, noting varied outcomes in natural domains, concluding with considerations for future research. The target paper, "Evaluation and Selection of Biases in Machine Learning," introduces and defines biases in machine learning, emphasizing the significance of automated methods for bias evaluation and selection through frameworks of bias/meta-bias spaces, summarizing recent research in this area.\r\rKeywords:\rSource paper: Case-based learning, Irrelevant features, Oblivion algorithm, Feature identification, Parity concepts, Experimental results\rTarget paper: Machine learning biases, Bias evaluation, Bias selec

In [9]:
for key in message.keys():
    h, cite_list = message[key]['h'][0], message[key]['w']

    cite_str = ""

    for i, w in enumerate(cite_list):
        cite_str += f"{i+1}. {w} \n"

    agg_prompt = f"""
Given the source and target papers, and information about the papers that cite the target paper:
source and target information:
{h}
the information about the paper cite the target paper:
{cite_str}
please summarize information of source and target papers and the citation relationship in no more than three sentences, and then generate some keywords of this information.
When outputting your answer, when referring to a paper, you should state the title of the paper and indicate whether it is a source paper or a target paper.
"""
    print(key)
    print(agg_prompt)

    print('============================================================================================================')

63486

Given the source and target papers, and information about the papers that cite the target paper:
source and target information:
Target paper: Machine learning biases, Bias evaluation, Bias selection, Meta-bias spaces, Automated methods, Research summaryntal results irrelevant features, proposing the Oblivion algorithm for efficient feature identification, especially with interacting features like parity concepts. It discusses experimental results in artificial domains supporting its hypothesis, noting varied outcomes in natural domains, concluding with considerations for future research. The target paper, "Evaluation and Selection of Biases in Machine Learning," introduces and defines biases in machine learning, emphasizing the significance of automated methods for bias evaluation and selection through frameworks of bias/meta-bias spaces, summarizing recent research in this area.
the information about the paper cite the target paper:
1. paper1:title:Title: Irrelevant Features an

In [16]:
# prediction

src_title = sample[sample.pid == src_id]['title'].values[0]


for i in range(len(sample)):
    if sample.loc[i, 'pid'] == src_id:
        continue

    tar_pid, tar_title = sample.loc[i, 'pid'], sample.loc[i, 'title']

    h1 = sample.loc[i, 'h1']
    print(src_id, tar_pid)


    prediction_prompt = f"""
given information of a source paper and a target paper shown as follow:
{h1}
Is there an existing citation relationship between "{src_title}" and "{tar_title}"
Do not give any reasoning or logic for your answer. Answer Yes if they have cited relationship, otherwise, answer No.
answer:

"""
    print(prediction_prompt)
    print('=========================')

10169 63486

given information of a source paper and a target paper shown as follow:
The source paper, "Learning Boolean Concepts in the Presence of Many Irrelevant Features," delves into case-based learning amid irrelevant features, introducing the Oblivion algorithm for efficient feature identification, particularly in scenarios involving interacting features like parity concepts. It presents experimental results supporting its thesis in artificial domains but highlights varying outcomes in natural domains and suggests avenues for future investigation. The target paper, "Evaluation and Selection of Biases in Machine Learning," defines biases in machine learning, emphasizing automated bias evaluation and selection using bias/meta-bias spaces, summarizing recent advancements in this domain. It has been cited by papers focusing on subset selection problems, robust feature selection algorithms, tactical plans enhancement with genetic algorithms, and leveraging prior concepts for learning